In [1]:
import os
import math
import json
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib.colors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from datetime import datetime
# from patsy import dmatrices
from patsy import dmatrix
from scipy.stats import ttest_rel
from scipy.stats import ttest_ind
import statsmodels.formula.api as smf

In [2]:
from causalinference import CausalModel
from causalinference.utils import random_data

In [3]:
Data_Root = '/Data/Promotion/'

In [4]:
CIs = {'90': 1.645, '95': 1.96, '99': 2.576}

In [5]:
labels = ['Male', 'Female']

In [6]:
colors = sns.color_palette()[:len(labels)]

In [7]:
reg_data = pd.read_csv(Data_Root+"revision/reg_data_drop_missing.csv", header=0, dtype={'matched_tid': str, \
        'matched_tid_retweet': str, 'matched_tid_original': str, 'pub_year': str})

In [8]:
reg_data = reg_data.loc[reg_data['gender'].isin(['Female', 'Male'])]
reg_data = reg_data.loc[reg_data.affiliation_cate != 'unknown']
reg_data.index = range(len(reg_data))

In [9]:
len(reg_data)

11396752

In [10]:
reg_data['author_citation_log'] = reg_data['author_citation'].apply(lambda x: np.log2(x+1))

In [11]:
reg_data['gender_CT'] = reg_data['gender'].apply(lambda gen: 1 if gen == 'Female' else 0)

In [42]:
co_feats_ = ["C(authorship_pos, Treatment(reference='last_position'))", 'author_pub_count_cate', 'affiliation_rank_cate', 'affiliation_cate', 'num_authors', \
         'journal_impact', 'author_citation_log', 'pub_year', \
        'Social_Sciences', 'Materials_Science', 'Engineering', 'Chemistry',
       'Biochemistry__Genetics_and_Molecular_Biology', 'Medicine', 'Nursing',
       'Agricultural_and_Biological_Sciences',
       'Pharmacology__Toxicology_and_Pharmaceutics', 'Neuroscience',
       'Business__Management_and_Accounting',
       'Economics__Econometrics_and_Finance', 'Chemical_Engineering',
       'Physics_and_Astronomy', 'Computer_Science', 'Decision_Sciences',
       'Health_Professions', 'Psychology', 'Immunology_and_Microbiology',
       'Dentistry', 'Earth_and_Planetary_Sciences', 'Environmental_Science',
       'Mathematics', 'Arts_and_Humanities', 'Energy', 'Veterinary', 'General']

In [43]:
X = dmatrix(formula_like=' + '.join(co_feats_), data=reg_data, return_type="dataframe")

In [44]:
list(X.columns)

['Intercept',
 "C(authorship_pos, Treatment(reference='last_position'))[T.first_position]",
 "C(authorship_pos, Treatment(reference='last_position'))[T.middle_position]",
 "C(authorship_pos, Treatment(reference='last_position'))[T.solo_author]",
 'affiliation_cate[T.international]',
 'pub_year[T.2014]',
 'pub_year[T.2015]',
 'pub_year[T.2016]',
 'pub_year[T.2017]',
 'pub_year[T.2018]',
 'author_pub_count_cate',
 'affiliation_rank_cate',
 'num_authors',
 'journal_impact',
 'author_citation_log',
 'Social_Sciences',
 'Materials_Science',
 'Engineering',
 'Chemistry',
 'Biochemistry__Genetics_and_Molecular_Biology',
 'Medicine',
 'Nursing',
 'Agricultural_and_Biological_Sciences',
 'Pharmacology__Toxicology_and_Pharmaceutics',
 'Neuroscience',
 'Business__Management_and_Accounting',
 'Economics__Econometrics_and_Finance',
 'Chemical_Engineering',
 'Physics_and_Astronomy',
 'Computer_Science',
 'Decision_Sciences',
 'Health_Professions',
 'Psychology',
 'Immunology_and_Microbiology',
 'Den

In [45]:
X['gender'] = reg_data['gender']
X['gender_CT'] = reg_data['gender_CT']
X['self_promotion'] = reg_data['self_promotion']

In [46]:
co_feats = [
 "C(authorship_pos, Treatment(reference='last_position'))[T.first_position]",
 "C(authorship_pos, Treatment(reference='last_position'))[T.middle_position]",
 "C(authorship_pos, Treatment(reference='last_position'))[T.solo_author]",
 'affiliation_cate[T.international]',
 'pub_year[T.2014]',
 'pub_year[T.2015]',
 'pub_year[T.2016]',
 'pub_year[T.2017]',
 'pub_year[T.2018]',
 'author_pub_count_cate',
 'affiliation_rank_cate',
 'num_authors',
 'journal_impact',
 'author_citation_log',
 'Social_Sciences',
 'Materials_Science',
 'Engineering',
 'Chemistry',
 'Biochemistry__Genetics_and_Molecular_Biology',
 'Medicine',
 'Nursing',
 'Agricultural_and_Biological_Sciences',
 'Pharmacology__Toxicology_and_Pharmaceutics',
 'Neuroscience',
 'Business__Management_and_Accounting',
 'Economics__Econometrics_and_Finance',
 'Chemical_Engineering',
 'Physics_and_Astronomy',
 'Computer_Science',
 'Decision_Sciences',
 'Health_Professions',
 'Psychology',
 'Immunology_and_Microbiology',
 'Dentistry',
 'Earth_and_Planetary_Sciences',
 'Environmental_Science',
 'Mathematics',
 'Arts_and_Humanities',
 'Energy',
 'Veterinary',
 'General']

In [47]:
# Y is the outcome, D is treatment status, and X is the independent variable
causal = CausalModel(Y=X['self_promotion'].values, D=X['gender_CT'].values, \
                     X=X[co_feats].values)

In [48]:
print(causal.summary_stats)


Summary Statistics

                   Controls (N_c=7371102)     Treated (N_t=4025650)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y        0.046        0.210        0.035        0.183       -0.011

                   Controls (N_c=7371102)     Treated (N_t=4025650)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0        0.151        0.358        0.194        0.395        0.112
             X1        0.640        0.480        0.658        0.474        0.037
             X2        0.017        0.128        0.013        0.114       -0.030
             X3        0.653        0.476        0.650        0.477       -0.006
             X4        0.150        0.357        0.148        0.355       -0.006
      

In [49]:
causal.est_propensity()

In [49]:
# Automated propensity score estimation of which features to use in the matching.
# causal.est_propensity_s()

In [50]:
# Propensity model results
print(causal.propensity)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      0.058      0.003     17.695      0.000      0.051      0.064
            X0      0.167      0.002     71.758      0.000      0.162      0.172
            X1      0.137      0.002     72.991      0.000      0.134      0.141
            X2     -0.191      0.006    -33.133      0.000     -0.202     -0.180
            X3      0.048      0.001     32.877      0.000      0.045      0.050
            X4      0.033      0.002     13.401      0.000      0.028      0.038
            X5      0.068      0.002     28.586      0.000      0.063      0.072
            X6      0.100      0.002     42.848      0.000      0.096      0.105
            X7      0.127      0.002     54.302      0.000      0.122      0.131
            X8      0.188      0.002     79.389      0.000      0.

In [51]:
causal.propensity['fitted']

array([0.22737879, 0.42781661, 0.34181751, ..., 0.29336662, 0.44319718,
       0.54685937])

### 1 on 1 matching using the nearest neighbor

Check covariate balance before matching

In [52]:
for feat in co_feats:
    print(feat)
    f_avg = np.mean(X.loc[X['gender']=='Female', feat])
    m_avg = np.mean(X.loc[X['gender']=='Male', feat])
    print('\tFemale:\t', f_avg)
    print('\tMale:\t', m_avg)
    print('\tDiff:\t', f_avg-m_avg)
    print('\tT-test:\t', ttest_ind(X.loc[X['gender']=='Female', feat], X.loc[X['gender']=='Male', feat])[1])

C(authorship_pos, Treatment(reference='last_position'))[T.first_position]
	Female:	 0.19351955584812391
	Male:	 0.15144709705550133
	Diff:	 0.042072458792622586
	T-test:	 0.0
C(authorship_pos, Treatment(reference='last_position'))[T.middle_position]
	Female:	 0.6580847813396594
	Male:	 0.6402958472152468
	Diff:	 0.017788934124412625
	T-test:	 0.0
C(authorship_pos, Treatment(reference='last_position'))[T.solo_author]
	Female:	 0.013133282823891795
	Male:	 0.016717581713019302
	Diff:	 -0.0035842988891275074
	T-test:	 0.0
affiliation_cate[T.international]
	Female:	 0.650354601120316
	Male:	 0.6530828090562306
	Diff:	 -0.0027282079359146616
	T-test:	 2.4062679946893132e-20
pub_year[T.2014]
	Female:	 0.14788270217232
	Male:	 0.14987067062699716
	Diff:	 -0.0019879684546771637
	T-test:	 2.1730750520364815e-19
pub_year[T.2015]
	Female:	 0.17185721560493336
	Male:	 0.1679662009832451
	Diff:	 0.0038910146216882546
	T-test:	 6.648496794265946e-63
pub_year[T.2016]
	Female:	 0.18450958230347894
	Ma

In [53]:
len(causal.propensity['fitted'])

11396752

In [54]:
X['pscore'] = causal.propensity['fitted']

In [55]:
tem = X[['gender', 'pscore']].sort_values(by = ['pscore'])
tem['index'] = tem.index

In [56]:
tem = tem.values.tolist()

In [57]:
tem[-10:]

[['Female', 0.8572876306515833, 3463197],
 ['Female', 0.8575073543545958, 4991019],
 ['Male', 0.8580432051602326, 5813196],
 ['Male', 0.8587982301788866, 5556808],
 ['Female', 0.85930477432767, 2470824],
 ['Female', 0.8593657657021483, 9668339],
 ['Female', 0.8593657657021483, 9668340],
 ['Female', 0.8593657657021483, 9668337],
 ['Female', 0.8593737654642198, 3176813],
 ['Female', 0.8610430115590644, 1008134]]

In [58]:
leng = len(tem)

In [59]:
pairs = {}

for i, elms in enumerate(tem):
    gen, score, ix = elms
    if gen == 'Female':
        pre_male_ix, nex_male_ix = 0, 0
        j = i-1
        while j >= 0:
            if tem[j][0] != 'Male':
                j -= 1
            else:
                break
        if j >= 0:
            pre_male_ix = j
        n = i+1
        while n <= leng-1:
            if tem[n][0] != 'Male':
                n += 1
            else:
                break
        if n <= leng-1:
            nex_male_ix = n
        if abs(score - tem[pre_male_ix][1]) <= abs(score - tem[nex_male_ix][1]):
            pairs[ix] = tem[pre_male_ix][2]
        else:
            pairs[ix] = tem[nex_male_ix][2]

In [60]:
del tem

In [61]:
len(pairs)

4025650

Check covariate balance after matching

In [63]:
for feat in co_feats:
    print('\n')
    print('Feat:', feat, '\n')
    print('Before matching:\n')
    f_avg = np.mean(X.loc[X['gender']=='Female', feat])
    m_avg = np.mean(X.loc[X['gender']=='Male', feat])
    print('\tFemale:\t', f_avg)
    print('\tMale:\t', m_avg)
    print('\tDiff:\t', f_avg-m_avg)
    print('\tT-test:\t', ttest_ind(X.loc[X['gender']=='Female', feat], X.loc[X['gender']=='Male', feat])[1])

    print('\nAfter matching:\n')
    f_avg = np.mean(X.loc[pairs.keys(), feat])
    m_avg = np.mean(X.loc[pairs.values(), feat])
    print('\tFemale:\t', f_avg)
    print('\tMale:\t', m_avg)
    print('\tDiff:\t', f_avg-m_avg)
    print('\tT-test:\t', ttest_rel(X.loc[pairs.keys(), feat], X.loc[pairs.values(), feat])[1])



Feat: C(authorship_pos, Treatment(reference='last_position'))[T.first_position] 

Before matching:

	Female:	 0.19351955584812391
	Male:	 0.15144709705550133
	Diff:	 0.042072458792622586
	T-test:	 0.0

After matching:

	Female:	 0.19351955584812391
	Male:	 0.1919215530411238
	Diff:	 0.0015980028070001207
	T-test:	 1.977331278342632e-09


Feat: C(authorship_pos, Treatment(reference='last_position'))[T.middle_position] 

Before matching:

	Female:	 0.6580847813396594
	Male:	 0.6402958472152468
	Diff:	 0.017788934124412625
	T-test:	 0.0

After matching:

	Female:	 0.6580847813396594
	Male:	 0.6613662389924608
	Diff:	 -0.0032814576528014117
	T-test:	 6.978046476618486e-24


Feat: C(authorship_pos, Treatment(reference='last_position'))[T.solo_author] 

Before matching:

	Female:	 0.013133282823891795
	Male:	 0.016717581713019302
	Diff:	 -0.0035842988891275074
	T-test:	 0.0

After matching:

	Female:	 0.013133282823891795
	Male:	 0.013176008843292387
	Diff:	 -4.272601940059212e-05
	T-test:

	T-test:	 0.027074892676966885


Feat: Computer_Science 

Before matching:

	Female:	 0.014188267733161104
	Male:	 0.026809966813646047
	Diff:	 -0.012621699080484943
	T-test:	 0.0

After matching:

	Female:	 0.014188267733161104
	Male:	 0.015082284848409573
	Diff:	 -0.0008940171152484692
	T-test:	 3.717098344693822e-28


Feat: Decision_Sciences 

Before matching:

	Female:	 0.002887732415883149
	Male:	 0.004968320883363166
	Diff:	 -0.0020805884674800165
	T-test:	 0.0

After matching:

	Female:	 0.002887732415883149
	Male:	 0.003044477289381839
	Diff:	 -0.00015674487349868994
	T-test:	 3.4710176437272315e-05


Feat: Health_Professions 

Before matching:

	Female:	 0.01080868927999205
	Male:	 0.01029045589112727
	Diff:	 0.0005182333888647805
	T-test:	 2.1376736710195383e-16

After matching:

	Female:	 0.01080868927999205
	Male:	 0.010712555736340714
	Diff:	 9.613354365133574e-05
	T-test:	 0.17809788466277013


Feat: Psychology 

Before matching:

	Female:	 0.039713835032851835
	Male:	 0.

In [64]:
for feat in co_feats:
    f_avg = np.mean(X.loc[X['gender']=='Female', feat])
    m_avg = np.mean(X.loc[X['gender']=='Male', feat])
    f_avg_ = np.mean(X.loc[pairs.keys(), feat])
    m_avg_ = np.mean(X.loc[pairs.values(), feat])
    print(feat, ' & ', '{:6.2f}'.format(f_avg), ' & ', '{:6.2f}'.format(m_avg), ' & ', '{:6.2f}'.format(f_avg_), ' & ', '{:6.2f}'.format(m_avg_), ' \\\\ \hline')

C(authorship_pos, Treatment(reference='last_position'))[T.first_position]  &    0.19  &    0.15  &    0.19  &    0.19  \\ \hline
C(authorship_pos, Treatment(reference='last_position'))[T.middle_position]  &    0.66  &    0.64  &    0.66  &    0.66  \\ \hline
C(authorship_pos, Treatment(reference='last_position'))[T.solo_author]  &    0.01  &    0.02  &    0.01  &    0.01  \\ \hline
affiliation_cate[T.international]  &    0.65  &    0.65  &    0.65  &    0.65  \\ \hline
pub_year[T.2014]  &    0.15  &    0.15  &    0.15  &    0.15  \\ \hline
pub_year[T.2015]  &    0.17  &    0.17  &    0.17  &    0.17  \\ \hline
pub_year[T.2016]  &    0.18  &    0.18  &    0.18  &    0.18  \\ \hline
pub_year[T.2017]  &    0.19  &    0.18  &    0.19  &    0.19  \\ \hline
pub_year[T.2018]  &    0.18  &    0.19  &    0.18  &    0.18  \\ \hline
author_pub_count_cate  &    4.08  &    5.41  &    4.08  &    4.04  \\ \hline
affiliation_rank_cate  &    4.35  &    4.28  &    4.35  &    4.38  \\ \hline
num_authors 

Avg treatment effect

In [65]:
# female avg
np.mean(X.loc[pairs.keys(), 'self_promotion'])

0.03473749580813036

In [66]:
# male avg
np.mean(X.loc[pairs.values(), 'self_promotion'])

0.04559909579819408

In [67]:
ttest_rel(X.loc[pairs.keys(), 'self_promotion'].apply(lambda x: 1 if x == True else 0), \
          X.loc[pairs.values(), 'self_promotion'].apply(lambda x: 1 if x == True else 0))

Ttest_relResult(statistic=-78.66955568226741, pvalue=0.0)

Avg treatment effect: `estimate via logit with covariates to control for the slight remaining imbalance`

In [205]:
# https://stats.stackexchange.com/questions/406379/how-exactly-to-evaluate-treatment-effect-after-matching

In [68]:
reg_data['self_promotion'] = reg_data['self_promotion'].astype('int32')

In [69]:
# eq = 'self_promotion ~ gender + ' + ' + '.join(co_feats_)

eq = "self_promotion ~ C(gender, Treatment(reference='Male')) + C(authorship_pos, Treatment(reference='last_position')) + author_pub_count_cate + \
    I(author_pub_count_cate**2) + affiliation_rank_cate + affiliation_cate + num_authors + journal_impact + author_citation_log + pub_year + \
        Social_Sciences + Materials_Science + Engineering + Chemistry + \
        Biochemistry__Genetics_and_Molecular_Biology + Medicine + Nursing + Agricultural_and_Biological_Sciences + \
        Pharmacology__Toxicology_and_Pharmaceutics + Neuroscience + Business__Management_and_Accounting + \
        Economics__Econometrics_and_Finance + Chemical_Engineering + Physics_and_Astronomy + Computer_Science + \
        Decision_Sciences + Health_Professions + Psychology + Immunology_and_Microbiology + Dentistry + \
        Earth_and_Planetary_Sciences + Environmental_Science + Mathematics + Arts_and_Humanities + Energy + \
        Veterinary + General"
model = smf.logit(formula = eq, data = reg_data.loc[list(pairs.keys()) + list(pairs.values())].sample(frac=1)).fit()

# for count DV.
# smf.glm(formula = eq, data=reg_data, family=sm.families.NegativeBinomial()).fit()

Optimization terminated successfully.
         Current function value: 0.152429
         Iterations 9


In [70]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         self_promotion   No. Observations:              8051300
Model:                          Logit   Df Residuals:                  8051256
Method:                           MLE   Df Model:                           43
Date:                Tue, 17 Jan 2023   Pseudo R-squ.:                 0.09526
Time:                        16:52:31   Log-Likelihood:            -1.2273e+06
converged:                       True   LL-Null:                   -1.3565e+06
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================================================================================
                                                                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                     -4.1737      0.012   -358.374      0.000      -4.197      -4.151
C(gender, Treatment(reference='Male'))[T.Female]                              -0.3125      0.004    -84.318      0.000      -0.320      -0.305
C(authorship_pos, Treatment(reference='last_position'))[T.first_position]      0.3377      0.006     58.720      0.000       0.326       0.349
C(authorship_pos, Treatment(reference='last_position'))[T.middle_position]    -0.6189      0.005   -117.296      0.000      -0.629      -0.609
C(authorship_pos, Treatment(reference='last_position'))[T.solo_author]         0.8041      0.011     74.371      0.000       0.783       0.825
affiliation_cate[T.international]                                              0.1065      0.004     25.801      0.000       0.098       0.115
pub_year[T.2014]                                                               0.3244      0.010     32.116      0.000       0.305       0.344
pub_year[T.2015]                                                               0.6569      0.009     70.029      0.000       0.639       0.675
pub_year[T.2016]                                                               0.8973      0.009     99.213      0.000       0.880       0.915
pub_year[T.2017]                                                               1.2603      0.009    144.004      0.000       1.243       1.277
pub_year[T.2018]                                                               1.6029      0.009    186.700      0.000       1.586       1.620
author_pub_count_cate                                                          0.2657      0.003     81.585      0.000       0.259       0.272
I(author_pub_count_cate ** 2)                                                 -0.0197      0.000    -67.093      0.000      -0.020      -0.019
affiliation_rank_cate                                                         -0.0567      0.001    -80.460      0.000      -0.058      -0.055
num_authors                                                                   -0.0007   2.52e-05    -26.470      0.000      -0.001      -0.001
journal_impact                                                                 0.0356      0.000    152.206      0.000       0.035       0.036
author_citation_log                                                           -0.0229      0.001    -22.039      0.000      -0.025      -0.021
Social_Sciences                                                                0.8339      0.008    110.277      0.000       0.819       0.849
Materials_Science                                                             -0.3325      0.018    -18.892      0.000      -0.367      -0.298
Engineering                                                                   -0.4413      0.01

In [71]:
print(model.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}                                                                & self\_promotion  & \textbf{  No. Observations:  } &   8051300    \\
\textbf{Model:}                                                                        &      Logit       & \textbf{  Df Residuals:      } &   8051256    \\
\textbf{Method:}                                                                       &       MLE        & \textbf{  Df Model:          } &        43    \\
\textbf{Date:}                                                                         & Tue, 17 Jan 2023 & \textbf{  Pseudo R-squ.:     } &   0.09526    \\
\textbf{Time:}                                                                         &     16:52:32     & \textbf{  Log-Likelihood:    } & -1.2273e+06  \\
\textbf{converged:}                                                                    &       True       & \textbf{  LL-Null:           } & -1.3565e+06  \\
\textbf{Cova

In [72]:
eq = "self_promotion ~ C(gender, Treatment(reference='Male'))"
model = smf.logit(formula = eq, data = reg_data.loc[list(pairs.keys()) + list(pairs.values())].sample(frac=1)).fit()

Optimization terminated successfully.
         Current function value: 0.168095
         Iterations 7


In [73]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         self_promotion   No. Observations:              8051300
Model:                          Logit   Df Residuals:                  8051298
Method:                           MLE   Df Model:                            1
Date:                Tue, 17 Jan 2023   Pseudo R-squ.:                0.002277
Time:                        16:53:51   Log-Likelihood:            -1.3534e+06
converged:                       True   LL-Null:                   -1.3565e+06
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================================================
                                                       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                           -3.0412      0.002  -1272.934      0.000      -3.046      -3.037
C(gender, Treatment(reference='Male'))[T.Female]    -0.2834      0.004    -78.248      0.000      -0.290      -0.276
====================================================================================================================
"""

In [74]:
# print(model.summary().as_latex())